# Script automatizado para extração de relatórios de programas de monitoramento analítico de produtos do Harpya

## Instalação e importação de bibliotecas necessárias

In [32]:
#Instalação de bibliotecas necessárias
#pip install xlrd
#pip install selenium

['Collecting xlrd',
 '  Downloading xlrd-2.0.1-py2.py3-none-any.whl (96 kB)',
 '     ---------------------------------------- 96.5/96.5 kB 2.7 MB/s eta 0:00:00',
 'Installing collected packages: xlrd',
 'Successfully installed xlrd-2.0.1']

In [19]:
#Carregamento de bibliotecas

import os
import pandas as pd
import shutil
import selenium.webdriver as drv
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from time import sleep
from getpass import getpass

## Criação de diretório base 

In [2]:
#Criação de pasta para salvar relatórios
diretorio = 'C:\Automação Harpya'

if not os.path.exists(diretorio):
    os.makedirs(diretorio)
    print(f"Diretório '{diretorio}' criado com sucesso.")
else:
    print(f"Diretório '{diretorio}' já existe.")

Diretório 'C:\Automação Harpya' já existe.


## Carregamento e manipulação de tabela base de filtros de buscas dos relatórios

In [3]:
#Carregamento tabela base de programas
base = pd.read_excel("C:/Automação Harpya/Programas.xlsx")

In [5]:
#Visualizar informações da tabela base
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Programa de monitoramento  3 non-null      object 
 1   Data inicial               3 non-null      object 
 2   Categoria de produto       3 non-null      object 
 3   Modalidade                 3 non-null      object 
 4   Produto                    2 non-null      object 
 5   URL Sharepoint             0 non-null      float64
dtypes: float64(1), object(5)
memory usage: 272.0+ bytes


In [6]:
#Substituir valores nulos por espaço em branco
base.fillna('', inplace=True)

In [13]:
#Criação de pasta para salvar relatórios de cada programa
raiz = 'C:/Automação Harpya/'

for index, row in base.iterrows():
    pasta = row['Programa de monitoramento']
    dir_pasta = raiz + pasta
    if not os.path.exists(dir_pasta):
        os.makedirs(dir_pasta)
        print(f"Diretório '{pasta}' criado com sucesso.")
    else:
        print(f"Diretório '{pasta}' já existe.")

Diretório 'Azitromicina' criado com sucesso.
Diretório 'Vacina' criado com sucesso.
Diretório 'Imunoglobulina' criado com sucesso.


## Captura de credenciais do usuário

In [20]:
#Inserção dos dados de login do usuário
usuario = input("Login: ")

Login: 


In [21]:
#Inserção da senha do usuário
senha = getpass("Senha: ")

Senha: ········


## Carregamento do navegador e acesso ao sistema Harpya

In [40]:
#Carregamento do navegador em modo anônimo
option = drv.ChromeOptions()
option.add_argument('-icognito')

navegador = drv.Chrome(options=option)

In [41]:
#Acesso à página do Harpya
navegador.get('https://harpya.datasus.gov.br/harpya/login.seam?cid=234690')

In [42]:
#Carregamento dos campos de usuário e senha e acesso ao sistema com a inserção das credenciais
campos = navegador.find_elements(By.CLASS_NAME, 'form-control')

sleep(1)

campos[0].send_keys(usuario)
campos[1].send_keys(senha)

sleep(1)

entrar = navegador.find_elements(By.ID, 'btnEntrar')
entrar[0].click()

## Extração dos relatórios e upload na pasta do Sharepoint

In [45]:
#Acesso à página de relatórios do Harpya
navegador.get('https://harpya.datasus.gov.br/harpya/views/gestao/nacional/home.seam?actionMethod=views%2Famostra.xhtml%3AgestaoLocal.iniciarGestaoNacional%28%27nacional%27%29&cid=234812&conversationPropagation=end')

In [46]:
#Seleção de campos para busca e extração dos relatórios

for index, row in base.iterrows():
    programa = row['Programa de monitoramento']
    data_in = row['Data inicial']
    categoria = row['Categoria de produto']
    modalidade = row['Modalidade']
    produto = row['Produto']
    url = row['URL Sharepoint']
    
    #Identificar campos e botão de buscar
    data_inicial = navegador.find_elements(By.ID, 'frmAmostras:tipoData1InputDate')
    cat_prod = Select(navegador.find_element(By.ID, 'frmAmostras:categoriaDeProduto'))
    mod = Select(navegador.find_element(By.ID, 'frmAmostras:modalidade'))
    prod = navegador.find_elements(By.ID, 'frmAmostras:nomeProduto')
    buscar = navegador.find_elements(By.ID, 'frmAmostras:btnBuscar')
    
    #Preenchimento dos dados
    data_inicial[0].clear()
    data_inicial[0].send_keys(data_in)
    cat_prod.select_by_visible_text(categoria)
    mod.select_by_visible_text(modalidade)
    prod[0].clear()
    prod[0].send_keys(produto)

    sleep(3)

    #Buscar relatório
    buscar[0].click()
    
    sleep(3)
    
    #Baixar relatório em Excel
    excel = navegador.find_elements(By.ID, 'frmResultados:btnExcel')
    excel[0].click()
    
    sleep(5)
    
    #Renomear e mover arquivo baixado
    caminho_antigo = "C:/Users/Danilo/Downloads/localizarAmostras-excel.xls"
    nome_arquivo = f"Harpya-{programa}.xls"
    novo_caminho = f"C:/Automação Harpya/{programa}/{nome_arquivo}"
    shutil.move(caminho_antigo, novo_caminho)
    
    #Converter o arquivo em xlsx
    arquivo = pd.read_excel(novo_caminho)
    arquivo.to_excel(f"{novo_caminho}x", index=False)
    
    # Excluir a versão antiga do arquivo - se enteder necessário
    #os.remove(novo_caminho)
    
    #Renomear e mover arquivo baixado para a pasta Sharepoint
    arquivo_xlsx = f"{novo_caminho}x"
    url_sharepoint = f"{url}/{nome_arquivo}x"
    shutil.copy(arquivo_xlsx, url_sharepoint)